# Datium Data Science Assessment – Section A
**Goal:** Predict vehicle sale prices (`Sold_Amount`).

This notebook follows a structured ML workflow:
1. Data loading & initial inspection
2. Exploratory Data Analysis (EDA)
3. Feature engineering & encoding
4. Model experimentation with MLflow tracking
5. Model evaluation & diagnostics
6. Final model selection

In [ ]:
import sys, os
sys.path.insert(0, os.path.join(os.getcwd(), '..', 'src'))

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import mlflow
import shap

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import Ridge
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

from trainer import VehiclePriceTrainer, BANNED_FEATURES, TARGET

pd.set_option('display.max_columns', 60)
pd.set_option('display.float_format', '{:.2f}'.format)
sns.set_theme(style='whitegrid', palette='muted')

SEED = 42
DATA_DIR = '../data'

## 1. Data Loading

In [ ]:
train_raw = pd.read_csv(f'{DATA_DIR}/train.rpt', sep='\t', low_memory=False, encoding='utf-8-sig')
test_raw  = pd.read_csv(f'{DATA_DIR}/test.rpt',  sep='\t', low_memory=False, encoding='utf-8-sig')

print(f'Train shape: {train_raw.shape}')
print(f'Test  shape: {test_raw.shape}')
train_raw.head(3)

## 2. Data Understanding & Quality Checks

In [ ]:
# ---- missing values ----
missing = train_raw.isnull().mean().sort_values(ascending=False)
missing_pct = (missing * 100).round(1)

fig, ax = plt.subplots(figsize=(10, 6))
missing_pct[missing_pct > 0].head(40).plot(kind='barh', ax=ax)
ax.set_title('Missing value rate by feature (train, top 40)')
ax.set_xlabel('% missing')
plt.tight_layout()
plt.savefig('../logs/missing_values.png', dpi=120)
plt.show()

print(f'Columns with >50% missing: {(missing_pct > 50).sum()}')
print(f'Columns with >90% missing: {(missing_pct > 90).sum()}')

In [ ]:
# ---- target distribution ----
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

train_raw[TARGET].dropna().plot(kind='hist', bins=60, ax=axes[0])
axes[0].set_title('Sold_Amount distribution')
axes[0].set_xlabel('Price ($)')

np.log1p(train_raw[TARGET].dropna()).plot(kind='hist', bins=60, ax=axes[1], color='steelblue')
axes[1].set_title('log1p(Sold_Amount) distribution')
axes[1].set_xlabel('log1p(Price)')

plt.tight_layout()
plt.savefig('../logs/target_distribution.png', dpi=120)
plt.show()

print(train_raw[TARGET].describe())

In [ ]:
# ---- outlier flag ----
q1, q99 = train_raw[TARGET].quantile([0.01, 0.99])
outliers = ((train_raw[TARGET] < q1) | (train_raw[TARGET] > q99)).sum()
print(f'Rows outside 1st–99th percentile: {outliers} ({outliers/len(train_raw)*100:.1f}%)')
print(f'1st pct: ${q1:,.0f}   99th pct: ${q99:,.0f}')

In [ ]:
# ---- duplicate check ----
print(f'Duplicate rows: {train_raw.duplicated().sum()}')

# ---- NULL string check (common in .rpt exports) ----
null_str_cols = [c for c in train_raw.columns if (train_raw[c] == 'NULL').any()]
print(f'Columns containing literal "NULL" string: {len(null_str_cols)}')

## 3. Feature Engineering

In [ ]:
def preprocess(df: pd.DataFrame, is_train: bool = True) -> pd.DataFrame:
    """Clean and engineer features."""
    df = df.copy()

    # Replace literal NULL strings with NaN
    df.replace('NULL', np.nan, inplace=True)

    # ---------- drop banned columns ----------
    df.drop(columns=[c for c in BANNED_FEATURES if c in df.columns], inplace=True)

    # ---------- drop columns with >70% missing ----------
    if is_train:
        global HIGH_MISS_COLS
        HIGH_MISS_COLS = df.columns[df.isnull().mean() > 0.70].tolist()
    df.drop(columns=[c for c in HIGH_MISS_COLS if c in df.columns], inplace=True)

    # ---------- date features ----------
    for col in ['Sold_Date', 'Compliance_Date']:
        if col in df.columns:
            df[col] = pd.to_datetime(df[col], errors='coerce')

    if 'Sold_Date' in df.columns:
        df['Sold_Year']  = df['Sold_Date'].dt.year
        df['Sold_Month'] = df['Sold_Date'].dt.month
        df['Sold_Quarter'] = df['Sold_Date'].dt.quarter

    # ---------- vehicle age at sale ----------
    if 'YearGroup' in df.columns and 'Sold_Date' in df.columns:
        df['Age_At_Sale'] = df['Sold_Date'].dt.year - pd.to_numeric(df['YearGroup'], errors='coerce')

    # ---------- KM per year ----------
    if 'KM' in df.columns and 'Age_Comp_Months' in df.columns:
        df['KM'] = pd.to_numeric(df['KM'], errors='coerce')
        df['Age_Comp_Months'] = pd.to_numeric(df['Age_Comp_Months'], errors='coerce')
        df['KM_Per_Month'] = df['KM'] / (df['Age_Comp_Months'].replace(0, np.nan))

    # ---------- engine size numeric ----------
    if 'EngineSize' in df.columns:
        df['EngineSize'] = pd.to_numeric(df['EngineSize'], errors='coerce')

    # ---------- NewPrice log ----------
    if 'NewPrice' in df.columns:
        df['NewPrice'] = pd.to_numeric(df['NewPrice'], errors='coerce')
        df['NewPrice_log'] = np.log1p(df['NewPrice'])
        df['Depreciation_Pct'] = 1 - (df['Sold_Amount'] / df['NewPrice'].replace(0, np.nan)) if is_train else np.nan

    # drop raw date columns
    df.drop(columns=['Sold_Date', 'Compliance_Date'], errors='ignore', inplace=True)

    return df


HIGH_MISS_COLS = []  # will be populated during train preprocessing
train_proc = preprocess(train_raw, is_train=True)
test_proc  = preprocess(test_raw,  is_train=False)

print(f'Processed train shape: {train_proc.shape}')
print(f'Processed test  shape: {test_proc.shape}')

In [ ]:
# ---- feature/target split ----
# Drop rows with missing target
train_clean = train_proc.dropna(subset=[TARGET]).copy()

# Use log-target for modelling (reduces skew impact)
y_log = np.log1p(train_clean[TARGET])

# Remove target + leakage columns from features
drop_for_features = [TARGET, 'Depreciation_Pct']
X = train_clean.drop(columns=[c for c in drop_for_features if c in train_clean.columns])

# ---- identify column types ----
num_cols = X.select_dtypes(include=[np.number]).columns.tolist()
cat_cols = X.select_dtypes(exclude=[np.number]).columns.tolist()

# Limit cardinality for ordinal encoding
high_card = [c for c in cat_cols if X[c].nunique() > 200]
cat_cols  = [c for c in cat_cols if c not in high_card]
X = X.drop(columns=high_card)

print(f'Numeric features : {len(num_cols)}')
print(f'Categorical features: {len(cat_cols)}')
print(f'Dropped high-cardinality: {high_card}')

X_train, X_val, y_train, y_val = train_test_split(X, y_log, test_size=0.2, random_state=SEED)

## 4. Preprocessing Pipeline

In [ ]:
numeric_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler',  StandardScaler()),
])

categorical_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)),
])

preprocessor = ColumnTransformer([
    ('num', numeric_transformer, num_cols),
    ('cat', categorical_transformer, cat_cols),
], remainder='drop')

print('Preprocessor ready.')

## 5. Model Experiments with MLflow Tracking

In [ ]:
mlflow.set_tracking_uri('../logs/mlruns')

MODELS = {
    'Ridge': Ridge(alpha=10.0),
    'RandomForest': RandomForestRegressor(n_estimators=200, max_depth=15, n_jobs=-1, random_state=SEED),
    'XGBoost': XGBRegressor(
        n_estimators=500, learning_rate=0.05, max_depth=6,
        subsample=0.8, colsample_bytree=0.8,
        n_jobs=-1, random_state=SEED, verbosity=0
    ),
    'LightGBM': LGBMRegressor(
        n_estimators=500, learning_rate=0.05, num_leaves=63,
        subsample=0.8, colsample_bytree=0.8,
        n_jobs=-1, random_state=SEED, verbose=-1
    ),
}

results = {}

for name, estimator in MODELS.items():
    print(f'\n===== {name} =====')
    pipe = Pipeline([('pre', preprocessor), ('model', estimator)])
    trainer = VehiclePriceTrainer(
        pipeline=pipe,
        experiment_name='vehicle_price',
        cv_folds=5,
        models_dir='../models',
    )
    trainer.fit(
        X_train, y_train,
        X_val=X_val, y_val=y_val,
        run_name=name,
        extra_tags={'model_type': name},
    )
    val_metrics = trainer.evaluate(X_val, y_val)
    results[name] = val_metrics

results_df = pd.DataFrame(results).T
print('\n--- Validation Results Summary ---')
print(results_df.sort_values('MAE'))

## 6. Model Diagnostics

In [ ]:
# Use best model (lowest MAE) for diagnostics
best_name = results_df['MAE'].idxmin()
print(f'Best model: {best_name}')

best_pipe = Pipeline([('pre', preprocessor), ('model', MODELS[best_name])])
best_pipe.fit(X_train, y_train)

val_pred_log = best_pipe.predict(X_val)
val_pred     = np.expm1(val_pred_log)   # back to dollars
val_true     = np.expm1(y_val.values)

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# -- Actual vs Predicted --
axes[0].scatter(val_true, val_pred, alpha=0.3, s=8)
mn, mx = val_true.min(), val_true.max()
axes[0].plot([mn, mx], [mn, mx], 'r--')
axes[0].set_xlabel('Actual ($)')
axes[0].set_ylabel('Predicted ($)')
axes[0].set_title('Actual vs Predicted')

# -- Residuals --
residuals = val_true - val_pred
axes[1].scatter(val_pred, residuals, alpha=0.3, s=8)
axes[1].axhline(0, color='r', linestyle='--')
axes[1].set_xlabel('Predicted ($)')
axes[1].set_ylabel('Residual ($)')
axes[1].set_title('Residuals vs Predicted')

# -- Residual distribution --
axes[2].hist(residuals, bins=60, edgecolor='k', alpha=0.7)
axes[2].set_xlabel('Residual ($)')
axes[2].set_title('Residual Distribution')

plt.tight_layout()
plt.savefig('../logs/diagnostics.png', dpi=120)
plt.show()

In [ ]:
# ---- Error by price bucket ----
buckets = pd.cut(val_true, bins=[0, 5000, 10000, 20000, 40000, 80000, np.inf],
                 labels=['<5k','5-10k','10-20k','20-40k','40-80k','>80k'])
bucket_df = pd.DataFrame({'true': val_true, 'pred': val_pred, 'bucket': buckets})
bucket_df['APE'] = np.abs(bucket_df['true'] - bucket_df['pred']) / bucket_df['true'].replace(0, np.nan)

bucket_summary = bucket_df.groupby('bucket', observed=True)['APE'].agg(['mean','count'])
bucket_summary.columns = ['MAPE', 'Count']
bucket_summary['MAPE'] = (bucket_summary['MAPE'] * 100).round(1)

print('Error by price bucket:')
print(bucket_summary)

bucket_summary['MAPE'].plot(kind='bar', figsize=(8,4), title='MAPE by Price Bucket (%)', rot=0)
plt.ylabel('MAPE (%)')
plt.tight_layout()
plt.savefig('../logs/error_by_bucket.png', dpi=120)
plt.show()

In [ ]:
# ---- Feature importance (SHAP) ----
X_val_transformed = best_pipe.named_steps['pre'].transform(X_val)
feature_names = num_cols + cat_cols

explainer = shap.TreeExplainer(best_pipe.named_steps['model'])
shap_values = explainer.shap_values(X_val_transformed)

plt.figure(figsize=(10, 8))
shap.summary_plot(shap_values, X_val_transformed, feature_names=feature_names, show=False, max_display=20)
plt.tight_layout()
plt.savefig('../logs/shap_summary.png', dpi=120)
plt.show()

## 7. Final Predictions on Test Set

In [ ]:
# Align test columns to training columns
X_test = test_proc.reindex(columns=X.columns)

# Fit final model on all training data
final_pipe = Pipeline([('pre', preprocessor), ('model', MODELS[best_name])])

X_all = pd.concat([X_train, X_val])
y_all = pd.concat([y_train, y_val])
final_pipe.fit(X_all, y_all)

test_pred_log = final_pipe.predict(X_test)
test_predictions = np.expm1(test_pred_log)

submission = pd.DataFrame({
    'Predicted_Sold_Amount': test_predictions.round(2)
})
submission.to_csv('../logs/predictions.csv', index=True)
print(f'Predictions saved. Shape: {submission.shape}')
print(submission['Predicted_Sold_Amount'].describe())

---
## Summary

| Step | Description |
|------|-------------|
| Data understanding | Checked missing rates, target distribution, outliers, duplicate rows, literal NULL strings |
| Feature engineering | Date decomposition, age at sale, KM/month, log-NewPrice, depreciation pct |
| Encoding | OrdinalEncoder for categoricals; median imputation for numerics |
| Models | Ridge, RandomForest, XGBoost, LightGBM |
| Evaluation | 5-fold CV (MAE/RMSE/R²), hold-out validation, residual plots, SHAP |
| Tracking | MLflow (metrics, params, artefacts) |
| Diagnostics | Residual patterns, error-by-price-bucket analysis |